# Espacio de Fases

Esta primera Notebook tiene como proposito presentar alguno de los sistemas que vimos en clase, calcular trayectorias (u orbitas) en el espacio de fases y estudiar como varian a medida que modificamos loa parametros del sistema. Vamos a presentar alguna de las herramientas que vamos a usar durante el curso, por ejemplo para integrar numericamente las ecuaciones diferenciales, graficar sus soluciones y usar interaccion para modificar los parametros y observar como cambian las soluciones en tiempo real.



In [ ]:
using Interact, Plots, DifferentialEquations

# Oscilador armonico
Sistema lineal que depende de un solo parametro K

Ecuaciones diferenciales de primer orden lineales correspondientes al oscilador armonico.
$x$ corresponde a la posicion del oscilador e $y$ a su velocidad.

$ \dot{x} = y $ 

$\dot{y} = -Kx$

La primera ecuacion solo especifica que la variacion temporal de la posicion corresponde a la velocidad. 

La segunda ecuacion, que proviene de la segunda Ley de Newton combinada con la Ley de Hooke especifica que la variacion de la velocidad (aceleracion) es propocional a la posicion con una constante $K>0$ y signo negativo

Si bien esta ecuación tiene solución analítica, vamos a integrarla numericamente usando el paquete **DifferentialEquations** de Julia. 

Para ello tenemos que definir primero la funcion que da las derivadas $du$ en function de las variables $u$ ($x=u[1]$, $y=u[2]$) y los parametros $p$, en este caso tenemos un unico parametro $K$ y podemos usar ese nombre pero de forma general vamos a tener un vector de parametros $p$.

In [ ]:
function oscharm!(du,u,K,t)
    du[1] = u[2]
    du[2] = -K*u[1]
end    

esta funcion en Julia se escribe con el bang **!** porque modifica sus argumentos (en este caso $du$)

## Calculo de las trayectorias
Luego tenemos que definir:
- una condicion inicial **u0** 
- un valor para el parámetro 
- un time span **tspan** (tiempo inicial y final)

con lo cual queda definido el problema **prob**

In [ ]:
u0 = [1.0;1.0]
K = 0.3
tspan = (0,100.0)
prob = ODEProblem(oscharm!, u0, tspan, K)

Para resolver simplemente apelamos a la funcion **solve** y guardamos la solucion en **sol**

In [ ]:
sol = solve(prob);

Finalmente graficamos la solucion en funcion del tiempo, y en el espacio de fases

In [ ]:
p1 = plot(sol)
p2 = plot(sol,vars=(1,2))
plot(p1,p2,layout=(1,2),size = (900,450))

## Interacción

Usando el macro @manipulate podemos generar un grafico interactivo

In [ ]:
@manipulate throttle=.05 for u1=-1:0.1:1,u2=-1:0.1:1,tmax=10:10:200,K=0:.1:3.0
    prob = ODEProblem(oscharm!, [u1; u2], (0, tmax), K)
    sol = solve(prob);
    p1 = plot(sol)
    p2 = plot(sol,vars=(1,2))
    scatter!(p2,[prob.u0[1]],[prob.u0[2]])
    plot(p1,p2,layout=(1,2),size = (900,450))
end

## Oscilador armonico con disipacion

Ahoera vamos a agregar un termino que de cuenta de la disipacion de energia que en su forma mas simple aparece como una fuerza con el signo contrario a la velocidad $y$ con una constante de proporcionalidad positiva $C$ y que se aplica a la segunda ecuación (Ley de Newton)

$ \dot{x} = y $ 

$\dot{y} = -Kx -Cy$

Notar que ahora como tenemos dos parametros vamos a introducirlos en la funcion como un vector $p=[K,C]$, y para mayuor claridad de notación hacemos un "unpack" de ese vector de parametros en la primera linea. El resto es igual.

In [ ]:
function oscharmdamp!(du,u,p,t)
    (K,C) = p
    du[1] = u[2]
    du[2] = -K*u[1]-C*u[2]
end    

In [ ]:
@manipulate throttle=.05 for u1=-1:0.1:1,u2=-1:0.1:1,tmax=10:10:200,K=0:.1:3.0,C=0:.02:1.0
    prob = ODEProblem(oscharmdamp!, [u1; u2], (0, tmax), [K,C])
    sol = solve(prob);
    p1 = plot(sol)
    p2 = plot(sol,vars=(1,2))
    scatter!(p2,[prob.u0[1]],[prob.u0[2]])
    plot(p1,p2,layout=(1,2),size = (900,450))
end

## Animacion

Usando el macro @animate podemos hacer un gif animado de la evolucion del sistema

In [ ]:
N = 100; dt = 0.5
sol = solve(ODEProblem(oscharmdamp!, [0.8; 0.8], (0, N*dt), [0.8,0.2]))
p1 = plot(sol,vars=(1,2),tspan=(0,dt),color=:black,xlims=(-1,1),ylims=(-1,1),legend=false,size=(400,400));
anim = @animate for n=2:N
    plot!(p1,sol,vars=(1,2),tspan=((n-1)*dt,n*dt),color=:black);
end;

In [ ]:
gif(anim, "oscharmdamp_anim_fps30.gif", fps = 30)

# Bouncing Ball

Este sistema es mas simple que el anterior porque si hacemos que $x$ se la altura de la pelota e $y$ su aceleracion, una pelota que cae por la acción de la gravedad se puede escribir simplemente 

$ \dot{x} = y $ 

$\dot{y} = -g$

la aceleracion es constante y negativa ($g=9.8$). 

Sin embargo, tenemos que agregar una condicion de rebote en el piso. Vamos a considerar dos casos: Rebote elastico (Ball Soft) y rebote rigido (Ball Hard). 

Para el caso del rebote elastico asumimos que cuando la altura llega a cero colisiona con un resorte de constante elastica $K$ que lo hace ir de vuelta hacia arriba (en este la pelota puede tomar valores negativos porque se deforma), esto implica que tenemos que evaluar el signo de $x$ y aplicar solo la aceleracion de la gravedad para $x>0$ y la gravedad mas la accion del resorte para $x<0$, esto implica una discontinuidad en el "campo vector" algo que no se lleva muy bien con los metodos de integracion numerica pero que en este caso Julia resuelve bastante bien:

$ \dot{x} = y $ 

$\dot{y} = -g$  [si $x>0$]

$ = -g-K*y$  [si $x<=0$]

In [ ]:
function ballsoft!(du,u,p,t)
    (g,K) = p
    du[1] = u[2]
    if (u[1]>0)
        du[2] = -g
    else
        du[2] = -g-K*u[1]
    end      
end

In [ ]:
u0 = [50.0,0.0]
tspan = (0.0,15.0)
p = [9.8, 20.0]
prob = ODEProblem(ballsoft!,u0,tspan,p)
sol = solve(prob)
p1 = plot(sol)
p2 = plot(sol,vars=(1,2))
plot(p1,p2,layout=(1,2),size = (900,450))

## Caso hard usando callback

El paquete DifferentialEquations permite introducir llamadas (callbacks) al integrador cuando se produce cierta condición y afectar entonces los parámetros o alguna de las variables. Esto es lo mas adecuado para introducir por ejemplo un cambio instantáneo signo de la velocidad como seria el caso del rebote perfectamente rígido. En este caso no hay ningun resorte y las ecuaciones son solo las de la caida de la pelota

In [ ]:
function ballhard!(du,u,p,t)
  du[1] = u[2]
  du[2] = -p[1]
end

Pero debemos introducir dos funciones nuevas. Una que chequee si la pelota toca el piso (la llamamos **infloor**) que por defecto se dispara cuando devuelve valor 0, con lo cual hacemos que devuelva el valor de $x$ ($u[1]$)

La otra función es la que se llama cuando se cumple la condicion anterior, la llamamos **bounce!** y lo que hace simplemente es cambiar el signo de la velocidad ($u[2]$)

In [ ]:
function infloor(u,t,integrator)
  u[1]
end

function bounce!(integrator)
  integrator.u[2] = -integrator.u[2]
end

Finalmente resolvemos nuestro problema pero agregando un callback que tiene como primer argumento a la condicion que debe cumplirse (**infloor**) y como segundo argumento la funcion a la que llama (**bounce!**)

In [ ]:
u0 = [50.0,0.0]
tspan = (0.0,15.0)
g = 9.8
prob = ODEProblem(ballhard!,u0,tspan,[g])
sol = solve(prob,callback=ContinuousCallback(infloor,bounce!);)
p1 = plot(sol)
p2 = plot(sol,vars=(1,2))
plot(p1,p2,layout=(1,2),size = (900,450))

## Bouncing Ball Hard con disipacion

Como caso final vamos a agregar disipacion en el rebote rigido. Aca asumimos que la disipacion de produce solo durante la colision y no por la friccion del aire (no seria dificil en ese caso solo hay que agegar como en el oscilador armonico un termino $-Cy$ en la segunda ecuacion).

En este caso la condicion es la misma se trata simplemente de cambiar la funcion callback para que en lugar de solo cambiar el signo de la velocidad la reduzca un poco, lo cual se logra multiplicandola por un valor menor que 1.0 con signo negativo.

Sin embargo, como cualquiera que vio una bola rebotando sabe, existe un potencial problema porque la cantidad de rebotes se va haciendo cada vez mayor y con intervalos de tiempo cada vez mas cortos. Para evitar que esto de problemas al integrador introducimos una condicion de corte cuando el paso de integracion (integrator.dt) se hace demasiado chico. Esa condicion de corte la introducimos en el nuevo callback **bouncedamped!**:

In [ ]:
function bouncedamped!(integrator)
  integrator.u[2] *= -0.9
  if integrator.dt > 1e-12
        set_proposed_dt!(integrator,(integrator.t-integrator.tprev)/100)
    else
        integrator.u[1] = 0
        integrator.u[2] = 0
        integrator.p[1] = 0
    end  
end

In [ ]:
u0 = [50.0,0.0]
tspan = (0.0,70.0)
g = 9.8
prob = ODEProblem(ballhard!,u0,tspan,[g])
sol = solve(prob,callback=ContinuousCallback(infloor,bouncedamped!))
p1 = plot(sol)
p2 = plot(sol,vars=(1,2))
plot(p1,p2,layout=(1,2),size = (900,450))

# Oscilador No Lineal : Van der Pol

Este es el primer sistema no lineal y lo vamos a ver en clase en mas detalle más adelante. Por ahora, podemos verlo como un oscilador donde la disipación es no lineal y en lugar de C es igual a $C(x^2-1)$
de forma tal que cuando x es menor que 1 la disipacion es **negativa** y actua como una fuente de energia


\begin{aligned}
\dot{x} & = y \\
\dot{y} & =  -Kx-Cy(x^2-1)  
\end{aligned}

In [ ]:
function vdp!(du,u,p,t)
    du[1] = u[2]
    du[2] = -p[1]*u[1]-p[2]*u[2]*(u[1]*u[1]-1.0)
    return # no devuelve nada
end    

In [ ]:
u0 = [1.0,1.0]
tspan = (0.0,30.0)
p = [1.0, 2.0]
prob = ODEProblem(vdp!,u0,tspan,p)
sol = solve(prob)
p1 = plot(sol)
p2 = plot(sol,vars=(1,2))
plot(p1,p2,layout=(1,2),size = (900,450))

In [ ]:
@manipulate throttle=.05 for u1=-1:0.1:1,u2=-1:0.1:1,tmax=10:10:200,K=0:.1:3.0,C=0:0.1:3.0
    prob = ODEProblem(vdp!, [u1; u2], (0, tmax), [K,C])
    sol = solve(prob);
    p1 = plot(sol)
    p2 = plot(sol,vars=(1,2))
    scatter!(p2,[prob.u0[1]],[prob.u0[2]])
    plot(p1,p2,layout=(1,2),size = (900,450))
end

In [ ]:
N = 50; dt = 0.5
sol = solve(ODEProblem(vdp!, [0.1; 0.1], (0, N*dt), [1.0,1.5]))
p1 = plot(sol,vars=(1,2),tspan=(0,dt),color=:black,xlims=(-2.2,2.2),ylims=(-4,4),legend=false,size=(400,400));
anim = @animate for n=2:N
    plot!(p1,sol,vars=(1,2),tspan=((n-1)*dt,n*dt),color=:black);
end;

In [ ]:
gif(anim, "../files/vanderpol_anim_fps30b.gif", fps = 30)

# Oscilador de Duffing 

Este es otro clásico oscilador no lineal y se puede interpretar como un oscilador en un pozo de potencial doble, pero por ahora lo usamos solo con fines expoloratorios. Tiene un parametro que corresponde a la disipación $\gamma$ y otro parámetro $\beta$ que si pasa de negativo a positivo cambia de tener un equilibrio estable a dos equilibriuos estables (doble pozo de potencial).

$\dot{x} = y$

$\dot{y} = -\gamma y + x (\beta -  x^2) $


In [ ]:
function duffing!(du,u,p,t)
    (γ,β) = p
    du[1] = u[2]
    du[2] = -γ*u[2]+u[1]*(β-u[1]*u[1])
    return
end    

In [ ]:
prob = ODEProblem(duffing!, [0.3; 1.0], (0, 50.0), [0.1,0.8])
sol = solve(prob)
p1 = plot(sol)
p2 = plot(sol,vars=(1,2),xlims=(-3, 3), ylims=(-2,2))
scatter!(p2,[prob.u0[1]],[prob.u0[2]])
plot(p1,p2,layout=(1,2),size = (900,300))

In [ ]:
@manipulate throttle=.05 for u1=-1:0.1:1,u2=-1:0.1:1,tmax=10:10:100,γ=0:.01:3.0,β=-1.0:0.1:3.0
    prob = ODEProblem(duffing!, [u1; u2], (0, tmax), [γ,β])
    sol = solve(prob);
    p1 = plot(sol)
    p2 = plot(sol,vars=(1,2),xlims=(-3, 3), ylims=(-2,2))
    scatter!(p2,[prob.u0[1]],[prob.u0[2]])
    plot(p1,p2,layout=(1,2),size = (900,300))
end

## Multiples trayectorias

Vamos a usar otra herramienta de DifferentialEquations que permite calcular muchas trayectorias simultaneamente (para eso es necesario tener activado el multithreading). 

Tenemos que definir primero un vector $u0\_arr$ con todas las condiciones iniciales, en este caso hacemos una grilla de 6x6 entre -1 y 1.

Luego definimos una funcion **prob_func** que recalcula nuestro problema con la nueva condicion inicial (la forma es estandar, lo que hay que tener presente es que el indice $i$ se refiere al numero de trayectoria)

In [ ]:
prob = ODEProblem(duffing!, [0.0; 0.0], (0, 50.0), [0.05,0.5])
u0_arr = vec([[0.2*i-0.5,0.2*j-0.5] for i=0:5, j=0:5])

function prob_func(prob,i,repeat)
  remake(prob,u0=u0_arr[i])
end

Finalmente definimos un **EnsembleProblem** con nuestro problema **prob** y nuestra funcion para redefinirlo **prob_func**, y resolvemos agregando como segundo argumento nuestro Ensemble y como tercer argumento el numero de trayectorias a calcular que hacemos igual al tamaño de nuestro vector de condiciones iniciales $u0\_arr$

In [ ]:
ensemble_prob = EnsembleProblem(prob,prob_func=prob_func)
@time sol = solve(ensemble_prob,EnsembleThreads(),trajectories=length(u0_arr))
plot(sol,vars=(1,2),size = (800,600))

In [ ]:
@manipulate throttle=.2 for tmax=10:10:100,γ=0:.02:1.0,β=-1.0:0.1:1.0,grid=3:8
    prob = ODEProblem(duffing!, [0.0; 0.0], (0, tmax), [γ,β])
    u0_arr = vec([[i/grid-0.5,j/grid-0.5] for i=0:grid, j=0:grid])
    function prob_func(prob,i,repeat)
      remake(prob,u0=u0_arr[i])
    end
    ensemble_prob = EnsembleProblem(prob,prob_func=prob_func)
    sol = solve(ensemble_prob,Tsit5(),EnsembleThreads(),trajectories=(length(u0_arr)))
    plot(sol,vars=(1,2),xlims=(-1.5, 1.5), ylims=(-1.0,1.0),size = (900,600))
end

# Oscilador de Duffing Forzado Periodicamente

$\dot{x} = y + A\cos (\omega t)$

$\dot{y} = -\gamma y + x(\beta -  x^2) $



In [ ]:
function duffingforced!(du,u,p,t)
    (γ,β,A,ω) = p
    du[1] = u[2] + A*cos(ω*t)
    du[2] = -γ*u[2]+u[1]*(β-u[1]*u[1])
    return
end    

In [ ]:
@manipulate throttle=.1 for tmax=10:10:1000,γ=0:.01:3.0,β=-1.0:0.1:3.0,A=0:0.02:1.0,ω=0:0.01:1.0
    prob = ODEProblem(duffingforced!, [0.2; 0.2], (0, tmax), [γ,β,A,ω])
    sol = solve(prob);
    plot(sol,vars=(1,2),size = (900,600))
end

# CHAOS!